In [1]:
import os
import torch
from torchvision import datasets, transforms
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data

In [2]:

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

path = 'dataset/'
dataset = torchvision.datasets.ImageFolder(root=path, transform=data_transforms)

# Split the dataset into train and test
train_size = int(0.8 * len(dataset) )
test_size = len(dataset) - train_size
train_dataset, test_dataset = data.random_split(dataset, [train_size, test_size],generator=torch.Generator().manual_seed(42))

# Define the dataloaders
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=32, shuffle=True)

trainloader=train_loader
testloader=test_loader


In [3]:
# print len of train and test data
print(len(train_dataset))
print(len(test_dataset))

737
185


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader

In [6]:
# Define ResNet model with modified first layer
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
# load model weights
model.load_state_dict(torch.load('resnet18.pth'))
model.cuda()
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

In [7]:

# Train the model
num_epochs = 15
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 10 == 9:
            print('[Epoch %d, Batch %d] loss: %.3f' % (epoch+1, i+1, running_loss / 10))
            running_loss = 0.0

[Epoch 1, Batch 10] loss: 1.487
[Epoch 1, Batch 20] loss: 0.455
[Epoch 2, Batch 10] loss: 0.302
[Epoch 2, Batch 20] loss: 0.287
[Epoch 3, Batch 10] loss: 0.181
[Epoch 3, Batch 20] loss: 0.309
[Epoch 4, Batch 10] loss: 0.250
[Epoch 4, Batch 20] loss: 0.229
[Epoch 5, Batch 10] loss: 0.255
[Epoch 5, Batch 20] loss: 0.160
[Epoch 6, Batch 10] loss: 0.132
[Epoch 6, Batch 20] loss: 0.139
[Epoch 7, Batch 10] loss: 0.167
[Epoch 7, Batch 20] loss: 0.125
[Epoch 8, Batch 10] loss: 0.156
[Epoch 8, Batch 20] loss: 0.153
[Epoch 9, Batch 10] loss: 0.242
[Epoch 9, Batch 20] loss: 0.184
[Epoch 10, Batch 10] loss: 0.212
[Epoch 10, Batch 20] loss: 0.126
[Epoch 11, Batch 10] loss: 0.272
[Epoch 11, Batch 20] loss: 0.217
[Epoch 12, Batch 10] loss: 0.164
[Epoch 12, Batch 20] loss: 0.171
[Epoch 13, Batch 10] loss: 0.177
[Epoch 13, Batch 20] loss: 0.182
[Epoch 14, Batch 10] loss: 0.158
[Epoch 14, Batch 20] loss: 0.108
[Epoch 15, Batch 10] loss: 0.170
[Epoch 15, Batch 20] loss: 0.151


In [8]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.cuda()
        labels = labels.cuda()

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        y_true += labels.cpu().numpy().tolist()
        y_pred += predicted.cpu().numpy().tolist()

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print('Accuracy: {:.2f}%'.format(accuracy * 100))
print('F1 Score: {:.2f}'.format(f1))

Accuracy: 94.05%
F1 Score: 0.94


In [24]:
# save model
torch.save(model.state_dict(), 'resnet18.pth')